In [16]:
import os
import gzip
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
from sim_model_archs import RCArch
from sim_model_archs import RegArch
from sim_model_archs import SiameseArch

Using TensorFlow backend.


In [3]:
from train_models import train_model 
from train_models import prepare_sequences
from train_models import save_all

In [4]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]= "1"

In [5]:
PARAMETERS = {
    'filters': 20,
    'kernel_size': 21,
    'input_length':200,
    'pool_size': 20, 
    'strides': 20
}

In [6]:
RC_WRAPPER = RCArch(**PARAMETERS)
REG_WRAPPER = RegArch(**PARAMETERS)
SIAMESE_WRAPPER = SiameseArch(**PARAMETERS)

In [22]:
!densityMotifSimulation.py --seed 1234 --motifNames ELF1_known2 --max-motifs 3 --min-motifs 1 --mean-motifs 2 --seqLength 200  --rc-prob 0.5 --numSeqs 10000

In [ ]:
!densityMotifSimulation.py --seed 1234 --motifNames GATA_known6 --max-motifs 3 --min-motifs 1 --mean-motifs 2 --seqLength 200 --numSeqs 10000 --rc-prob 0.5

In [ ]:
# !densityMotifSimulation.py --seed 1234 --motifNames RXRA_known1 --max-motifs 3 --min-motifs 1 --mean-motifs 2 --seqLength 200 --numSeqs 10000 --rc-prob 0.5

In [23]:
x_train, x_test, y_train, y_train_mutate, y_test = prepare_sequences(seq_len = 200)

In [27]:
y_train

array([[1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       ...,
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.]])

In [7]:
x_train_200, x_val_200, x_test_200, y_train_200, y_train_200_mutate, y_val_200, y_test_200 = prepare_sequences(seq_len = 200)

In [8]:
# x_train_1000, x_val_1000, x_test_1000, y_train_1000, y_train_1000_mutate, y_val_1000, y_test_1000 = prepare_sequences(seq_len = 1000)

In [7]:
all_seeds = [1535, 9999, 4652, 6806, 7626, 2143, 3578, 1780, 6117, 9295]

In [8]:
# !pip install Bio
from Bio import SeqIO
fasta_sequences = SeqIO.parse(open("simdata.fasta"),'fasta')
x_curr = []
for fasta in fasta_sequences:
    name, sequence = fasta.id, str(fasta.seq)
    x_curr.append(sequence)

In [9]:
x_curr

['...',
 ...]

In [10]:
ltrdict = {'a':[1,0,0,0],
           'c':[0,1,0,0],
           'g':[0,0,1,0],
           't':[0,0,0,1],
           'n':[0,0,0,0],
           'A':[1,0,0,0],
           'C':[0,1,0,0],
           'G':[0,0,1,0],
           'T':[0,0,0,1],
           'N':[0,0,0,0]}

def onehot_encode(sequence):
    return np.array([ltrdict[x] for x in sequence])

x_curr = np.array([onehot_encode(curr_seq) for curr_seq in x_curr])

In [11]:
import csv
y_curr = np.zeros((0,3))
with open("simdata.labels", "r") as f: 
    reader = csv.reader(f, delimiter = "\t")
    d = list(reader)
    curr_num = 0
    for thing in range(1, len(d)):
        curr = d[thing]
        other = []
        for x in range(1, 4): 
            other.append(float(curr[x]))
        y_curr = np.append(y_curr, [other], axis = 0)

In [12]:
x_train, x_test, y_train, y_test = train_test_split(x_curr, y_curr, test_size = 0.3, random_state = 1000)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 3/7, random_state = 1000)

In [13]:
len(x_train)

12000

In [18]:
# def mutate(y, mutation_prob):
#     np.random.seed(1234)
#     mutated_y = []
#     for row in y:
#         new_labels_for_row = []
#         for label in row:
#             if np.random.uniform() < mutation_prob:
#                 new_labels_for_row.append(1-label)
#             else:
#                 new_labels_for_row.append(label)
#         mutated_y.append(new_labels_for_row)
#     return np.array(mutated_y)

# y_train_mutate = mutate(y_train, 0.2)

In [10]:
# import numpy as np 
# import tensorflow as tf 
# import keras_genomics
# import tensorflow as tf
# import keras 
# import keras_genomics
# import numpy as np
# import os
# import gzip

# from numpy.random import seed
# from tensorflow import set_random_seed
# from keras.callbacks import EarlyStopping, History, ModelCheckpoint
# from sklearn.metrics import roc_auc_score
# from sklearn.metrics import average_precision_score
# from simdna.synthetic.core import read_simdata_file
# from sklearn.model_selection import train_test_split

# def train_model_other(model_wrapper, aug, curr_seed, batch_size, x, y):
#     np.random.seed(curr_seed)
#     tf.set_random_seed(curr_seed)
    
#     model = model_wrapper.get_model()
    
#     if aug == "rev_after_each": 
#         x_train = np.asarray([val for val in x for __ in (0,1)])
#         y_train = np.asarray([val for val in y for __ in (0,1)])
#         for i in range(len(x_train)):
#             if i % 2 == 1:
#                 x_train[i] = np.flip(x_train[i])
#     elif aug == "rev_after_all":
#         x_train = np.concatenate([x,x])
#         y_train = np.concatenate([y,y])
#         for i in range(len(x/2) + 1, len(x)):
#             x_train[i] = np.flip(x_train[i])
#     else: 
#         x_train = x
#         y_train = y
        
#     early_stopping_callback = keras.callbacks.EarlyStopping(
#                               monitor='val_loss',
#                               patience = 10,
#                               restore_best_weights=True)
    
#     model.fit(x = x_train, y = y_train, validation_split = 3/7,  
#               callbacks =[early_stopping_callback], 
#               batch_size=batch_size, epochs=200)
    
#     return model, early_stopping_callback


In [14]:
for curr_seed in all_seeds: 
    model, early_stopping_callback, auroc_callback = train_model(model_wrapper = RC_WRAPPER, 
                                                                 aug = None, 
                                                                 curr_seed = curr_seed,
                                                                 batch_size = 500,
                                                                 x = x_train, 
                                                                 y = y_train, 
                                                                 val_data = (x_val, y_val))
    save_all(filepath = "general_results_again", model_arch = "exact_RC_200_auROC", curr_seed = curr_seed,
             callback = auroc_callback, model = model, val_data = (x_val, y_val))
    save_all(filepath = "general_results_again", model_arch = "exact_RC_200_loss", curr_seed = curr_seed,
             callback = early_stopping_callback, model = model, val_data = (x_val, y_val))








Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 12000 samples, validate on 9000 samples
Epoch 1/200





12000/12000 [==============================] - 4s 333us/step - loss: 0.7585 - acc: 0.6032 - val_loss: 0.6778 - val_acc: 0.6562
Epoch 2/200
12000/12000 [==============================] - 0s 18us/step - loss: 0.6382 - acc: 0.6540 - val_loss: 0.6293 - val_acc: 0.6656
Epoch 3/200
12000/12000 [==============================] - 0s 18us/step - loss: 0.6016 - acc: 0.6853 - val_loss: 0.6016 - val_acc: 0.6870
Epoch 4/200
12000/12000 [==============================] - 0s 18us/step - loss: 0.5724 - acc: 0.7073 - val_loss: 0.5802 - val_acc: 0.7016
Epoch 5/200
12000/12000 [==============================] - 0s 18us/step - loss: 0.5487 - acc: 0.7256 - val_loss: 0.5600 - val_acc: 0.7177
Epoch 6/200
12000/12000 [==============================] - 0s 18us/step - loss: 0.5287 - acc: 0.7380 - val_loss: 0.5420 - val_acc: 0.7313
Epoch 7/200
1

In [15]:
for curr_seed in all_seeds: 
    model, early_stopping_callback, auroc_callback = train_model(model_wrapper = REG_WRAPPER, 
                                                                 aug = None, 
                                                                 curr_seed = curr_seed,
                                                                 batch_size = 500,
                                                                 x = x_train, 
                                                                 y = y_train, 
                                                                 val_data = (x_val, y_val))
    save_all(filepath = "general_results_again", model_arch = "exact_Reg_200_auROC", curr_seed = curr_seed,
             callback = auroc_callback, model = model, val_data = (x_val, y_val))
    save_all(filepath = "general_results_again", model_arch = "exact_Reg_200_loss", curr_seed = curr_seed,
             callback = early_stopping_callback, model = model, val_data = (x_val, y_val))

Train on 12000 samples, validate on 9000 samples
Epoch 1/200
12000/12000 [==============================] - 2s 137us/step - loss: 0.7585 - acc: 0.6032 - val_loss: 0.6778 - val_acc: 0.6562
Epoch 2/200
12000/12000 [==============================] - 0s 19us/step - loss: 0.6382 - acc: 0.6540 - val_loss: 0.6293 - val_acc: 0.6656
Epoch 3/200
12000/12000 [==============================] - 0s 19us/step - loss: 0.6016 - acc: 0.6853 - val_loss: 0.6016 - val_acc: 0.6871
Epoch 4/200
12000/12000 [==============================] - 0s 19us/step - loss: 0.5724 - acc: 0.7073 - val_loss: 0.5802 - val_acc: 0.7014
Epoch 5/200
12000/12000 [==============================] - 0s 19us/step - loss: 0.5487 - acc: 0.7256 - val_loss: 0.5600 - val_acc: 0.7177
Epoch 6/200
12000/12000 [==============================] - 0s 19us/step - loss: 0.5287 - acc: 0.7380 - val_loss: 0.5420 - val_acc: 0.7312
Epoch 7/200
12000/12000 [==============================] - 0s 19us/step - loss: 0.5129 - acc: 0.7494 - val_loss: 0.5295 - 

KeyboardInterrupt: 

In [14]:
for curr_seed in all_seeds: 
    model, early_stopping_callback = train_model_other(model_wrapper = RC_WRAPPER, 
                                                                 aug = None, 
                                                                 curr_seed = curr_seed,
                                                                 batch_size = 500,
                                                                 x = x_train, 
                                                                 y = y_train_mutate)
#     save_all(filepath = "general_results_again", model_arch = "split_RC_200_auROC", curr_seed = curr_seed,
#              callback = auroc_callback, model = model, val_data = (x_test, y_test))
    save_all(filepath = "general_results_again", model_arch = "split_RC_200_loss", curr_seed = curr_seed,
             callback = early_stopping_callback, model = model, val_data = (x_test, y_test))

Train on 12000 samples, validate on 9000 samples
Epoch 1/200
12000/12000 [==============================] - 1s 67us/step - loss: 0.7956 - acc: 0.5416 - val_loss: 0.7209 - val_acc: 0.5772
Epoch 2/200
12000/12000 [==============================] - 0s 18us/step - loss: 0.6997 - acc: 0.5719 - val_loss: 0.6967 - val_acc: 0.5867
Epoch 3/200
12000/12000 [==============================] - 0s 18us/step - loss: 0.6758 - acc: 0.5984 - val_loss: 0.6824 - val_acc: 0.5953
Epoch 4/200
12000/12000 [==============================] - 0s 17us/step - loss: 0.6598 - acc: 0.6157 - val_loss: 0.6722 - val_acc: 0.6018
Epoch 5/200
12000/12000 [==============================] - 0s 18us/step - loss: 0.6479 - acc: 0.6277 - val_loss: 0.6655 - val_acc: 0.6115
Epoch 6/200
12000/12000 [==============================] - 0s 18us/step - loss: 0.6384 - acc: 0.6397 - val_loss: 0.6589 - val_acc: 0.6219
Epoch 7/200
12000/12000 [==============================] - 0s 17us/step - loss: 0.6304 - acc: 0.6499 - val_loss: 0.6522 - v

KeyboardInterrupt: 

In [ ]:
for curr_seed in all_seeds: 
    model, early_stopping_callback, auroc_callback = train_model(model_wrapper = RC_WRAPPER, 
                                                                 aug = None, 
                                                                 curr_seed = curr_seed,
                                                                 batch_size = 500,
                                                                 x = x_train, 
                                                                 y = y_train_mutate, 
                                                                 val_data = (x_val_200, y_val_200))
    save_all(filepath = "general_results_again", model_arch = "test_RC_200_auROC", curr_seed = curr_seed,
             callback = auroc_callback, model = model, val_data = (x_test_200, y_test_200))
    save_all(filepath = "general_results_again", model_arch = "test_RC_200_loss", curr_seed = curr_seed,
             callback = early_stopping_callback, model = model, val_data = (x_test_200, y_test_200))

In [20]:
for curr_seed in all_seeds: 
    model, early_stopping_callback, auroc_callback = train_model(model_wrapper = RC_WRAPPER, 
                                                                 aug = None, 
                                                                 curr_seed = curr_seed,
                                                                 batch_size = 500,
                                                                 x = x_train_200, 
                                                                 y = y_train_200_mutate, 
                                                                 val_data = (x_val_200, y_val_200))
    save_all(filepath = "general_results_again", model_arch = "test_RC_200_auROC", curr_seed = curr_seed,
             callback = auroc_callback, model = model, val_data = (x_test_200, y_test_200))
    save_all(filepath = "general_results_again", model_arch = "test_RC_200_loss", curr_seed = curr_seed,
             callback = early_stopping_callback, model = model, val_data = (x_test_200, y_test_200))

Train on 12000 samples, validate on 9000 samples
Epoch 1/200
12000/12000 [==============================] - 5s 381us/step - loss: 0.7555 - acc: 0.5449 - val_loss: 0.6699 - val_acc: 0.6184
Epoch 2/200
12000/12000 [==============================] - 0s 21us/step - loss: 0.7017 - acc: 0.5734 - val_loss: 0.6651 - val_acc: 0.6037
Epoch 3/200
12000/12000 [==============================] - 0s 21us/step - loss: 0.6835 - acc: 0.5868 - val_loss: 0.6440 - val_acc: 0.6293
Epoch 4/200
12000/12000 [==============================] - 0s 22us/step - loss: 0.6682 - acc: 0.6019 - val_loss: 0.6268 - val_acc: 0.6464
Epoch 5/200
12000/12000 [==============================] - 0s 21us/step - loss: 0.6550 - acc: 0.6160 - val_loss: 0.6095 - val_acc: 0.6648
Epoch 6/200
12000/12000 [==============================] - 0s 21us/step - loss: 0.6426 - acc: 0.6311 - val_loss: 0.5955 - val_acc: 0.6788
Epoch 7/200
12000/12000 [==============================] - 0s 21us/step - loss: 0.6318 - acc: 0.6437 - val_loss: 0.5704 - 

In [18]:
for curr_seed in all_seeds: 
    model, early_stopping_callback, auroc_callback = train_model(model_wrapper = REG_WRAPPER, 
                                                                 aug = None, 
                                                                 curr_seed = curr_seed, 
                                                                 x = x_train_200, 
                                                                 y = y_train_200_mutate, 
                                                                 batch_size = 500,
                                                                 val_data = (x_val_200, y_val_200))
    save_all(filepath = "general_results_again", model_arch = "test_Standard_200_auROC", curr_seed = curr_seed,
             callback = auroc_callback, model = model, val_data = (x_test_200, y_test_200))
    save_all(filepath = "general_results_again", model_arch = "test_Standard_200_loss", curr_seed = curr_seed,
             callback = early_stopping_callback, model = model, val_data = (x_test_200, y_test_200))
    
#     model, early_stopping_callback, auroc_callback = train_model(model_wrapper = REG_WRAPPER, 
#                                                                  aug = "rev_after_each", 
#                                                                  curr_seed = curr_seed, 
#                                                                  x = x_train_200, 
#                                                                  y = y_train_200_mutate, 
#                                                                  val_data = (x_val_200, y_val_200))
#     save_all(filepath = "general_results", model_arch = "Aug_200_auROC", curr_seed = curr_seed,
#              callback = auroc_callback, model = model, val_data = (x_val_200, y_val_200))
#     save_all(filepath = "general_results", model_arch = "Aug_200_loss", curr_seed = curr_seed,
#              callback = early_stopping_callback, model = model, val_data = (x_val_200, y_val_200))
    
#     model, early_stopping_callback, auroc_callback = train_model(model_wrapper = REG_WRAPPER, 
#                                                                  aug = "rev_after_all", 
#                                                                  curr_seed = curr_seed, 
#                                                                  x = x_train_200, 
#                                                                  y = y_train_200_mutate, 
#                                                                  val_data = (x_val_200, y_val_200))
#     save_all(filepath = "general_results", model_arch = "Aug_Alt_200_auROC", curr_seed = curr_seed,
#              callback = auroc_callback, model = model, val_data = (x_val_200, y_val_200))
#     save_all(filepath = "general_results", model_arch = "Aug_Alt_200_loss", curr_seed = curr_seed,
#              callback = early_stopping_callback, model = model, val_data = (x_val_200, y_val_200))

#     model, early_stopping_callback, auroc_callback = train_model(model_wrapper = REG_WRAPPER, 
#                                                                  aug = "rev_after_all", 
#                                                                  curr_seed = curr_seed,
#                                                                  batch_size = 1000,
#                                                                  x = x_train_200, 
#                                                                  y = y_train_200_mutate, 
#                                                                  val_data = (x_val_200, y_val_200))
#     save_all(filepath = "general_results", model_arch = "Aug_Alt_double_200_auROC", curr_seed = curr_seed,
#              callback = auroc_callback, model = model, val_data = (x_val_200, y_val_200))
#     save_all(filepath = "general_results", model_arch = "Aug_Alt_double_200_loss", curr_seed = curr_seed,
#              callback = early_stopping_callback, model = model, val_data = (x_val_200, y_val_200))

Train on 12000 samples, validate on 9000 samples
Epoch 1/200
12000/12000 [==============================] - 3s 257us/step - loss: 0.7947 - acc: 0.5394 - val_loss: 0.6699 - val_acc: 0.6231
Epoch 2/200
12000/12000 [==============================] - 0s 20us/step - loss: 0.7019 - acc: 0.5668 - val_loss: 0.6466 - val_acc: 0.6300
Epoch 3/200
12000/12000 [==============================] - 0s 20us/step - loss: 0.6790 - acc: 0.5928 - val_loss: 0.6278 - val_acc: 0.6479
Epoch 4/200
12000/12000 [==============================] - 0s 21us/step - loss: 0.6636 - acc: 0.6108 - val_loss: 0.6034 - val_acc: 0.6760
Epoch 5/200
12000/12000 [==============================] - 0s 21us/step - loss: 0.6504 - acc: 0.6271 - val_loss: 0.5922 - val_acc: 0.6860
Epoch 6/200
12000/12000 [==============================] - 0s 21us/step - loss: 0.6392 - acc: 0.6377 - val_loss: 0.5781 - val_acc: 0.7000
Epoch 7/200
12000/12000 [==============================] - 0s 20us/step - loss: 0.6299 - acc: 0.6510 - val_loss: 0.5626 - 

In [13]:
for curr_seed in all_seeds: 
    model, early_stopping_callback, auroc_callback = train_model(model_wrapper = SIAMESE_WRAPPER, 
                                                                 aug = None, 
                                                                 curr_seed = curr_seed, 
                                                                 x = x_train_200, 
                                                                 y = y_train_200_mutate, 
                                                                 val_data = (x_val_200, y_val_200))
    save_all(filepath = "general_results", model_arch = "Siamese_200_auROC", curr_seed = curr_seed,
             callback = auroc_callback, model = model, val_data = (x_val_200, y_val_200))
    save_all(filepath = "general_results", model_arch = "Siamese_200_loss", curr_seed = curr_seed,
             callback = early_stopping_callback, model = model, val_data = (x_val_200, y_val_200))

Train on 17142 samples, validate on 12858 samples
Epoch 1/200
17142/17142 [==============================] - 6s 339us/step - loss: 0.6994 - acc: 0.6273 - val_loss: 0.6046 - val_acc: 0.6791
Epoch 2/200
17142/17142 [==============================] - 0s 28us/step - loss: 0.5705 - acc: 0.7030 - val_loss: 0.5420 - val_acc: 0.7239
Epoch 3/200
17142/17142 [==============================] - 0s 27us/step - loss: 0.5067 - acc: 0.7585 - val_loss: 0.4826 - val_acc: 0.7678
Epoch 4/200
17142/17142 [==============================] - 0s 27us/step - loss: 0.4527 - acc: 0.7984 - val_loss: 0.4354 - val_acc: 0.8039
Epoch 5/200
17142/17142 [==============================] - 0s 27us/step - loss: 0.4079 - acc: 0.8295 - val_loss: 0.3931 - val_acc: 0.8343
Epoch 6/200
17142/17142 [==============================] - 0s 27us/step - loss: 0.3713 - acc: 0.8488 - val_loss: 0.3594 - val_acc: 0.8509
Epoch 7/200
17142/17142 [==============================] - 0s 27us/step - loss: 0.3410 - acc: 0.8684 - val_loss: 0.3315 -

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Epoch 36/200
17142/17142 [==============================] - 1s 30us/step - loss: 0.1554 - acc: 0.9450 - val_loss: 0.1723 - val_acc: 0.9331
Epoch 37/200
17142/17142 [==============================] - 0s 29us/step - loss: 0.1538 - acc: 0.9463 - val_loss: 0.1693 - val_acc: 0.9339
Epoch 38/200
17142/17142 [==============================] - 0s 29us/step - loss: 0.1530 - acc: 0.9463 - val_loss: 0.1668 - val_acc: 0.9356
Epoch 39/200
17142/17142 [==============================] - 0s 28us/step - loss: 0.1509 - acc: 0.9477 - val_loss: 0.1672 - val_acc: 0.9353
Epoch 40/200
17142/17142 [==============================] - 0s 29us/step - loss: 0.1490 - acc: 0.9483 - val_loss: 0.1648 - val_acc: 0.9362
Epoch 41/200
17142/17142 [==============================] - 0s 28us/step - loss: 0.1487 - acc: 0.9479 - val_loss: 0.1671 - val_acc: 0.9348
Epoch 42/200
17142/17142 [==============================] - 0s 29us/step - loss: 0.1465 - acc: 0.9494 - val_loss: 0.1628 - val_acc: 0.9370
Epoch 43/200
17142/17142 [=

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [13]:
PARAMETERS = {
    'filters': 20,
    'kernel_size': 21,
    'input_length':200,
    'pool_size': 20, 
    'strides': 20
}

In [14]:
RC_WRAPPER = RCArch(**PARAMETERS)
REG_WRAPPER = RegArch(**PARAMETERS)
SIAMESE_WRAPPER = SiameseArch(**PARAMETERS)

In [21]:
from keras import backend as K 

import keras.layers as kl
from keras.layers import Input
from keras.layers.core import Dropout 
from keras.layers.core import Flatten

from keras.engine import Layer
from keras.engine.base_layer import InputSpec

from keras.models import Sequential 
from keras.models import Model
from keras.models import load_model
from keras_genomics.layers import RevCompConv1D

from numpy.random import seed
from tensorflow import set_random_seed
from keras.callbacks import EarlyStopping, History, ModelCheckpoint
  
    
class RevCompSumPool(Layer): 
    def __init__(self, **kwargs): 
        super(RevCompSumPoolAvg, self).__init__(**kwargs)

    def build(self, input_shape):
        self.num_input_chan = input_shape[2]
        super(RevCompSumPoolAvg, self).build(input_shape)

    def call(self, inputs): 
        inputs = (inputs[:,:,:int(self.num_input_chan/2)] + inputs[:,:,int(self.num_input_chan/2):][:,::-1,::-1])
        return inputs
      
    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[1], int(input_shape[2]/2))

In [29]:
PARAMETERS = {
    'filters': 20,
    'kernel_size': 21,
    'input_length':1000,
    'pool_size': 20, 
    'strides': 20
}

In [ ]:
from keras.utils import CustomObjectScope
from keras.models import load_model
with CustomObjectScope({'RevCompSumPool': RevCompSumPool}):
    loaded_model = load_model('MyProfileModel.h5')
    for model in model_names:

In [30]:
RC_WRAPPER = RCArch(**PARAMETERS)
REG_WRAPPER = RegArch(**PARAMETERS)
SIAMESE_WRAPPER = SiameseArch(**PARAMETERS)

In [19]:
for curr_seed in all_seeds: 
    model, early_stopping_callback, auroc_callback = train_model(model_wrapper = REG_WRAPPER, 
                                                                 aug = "rev_after_all", 
                                                                 curr_seed = curr_seed,
                                                                 batch_size = 1000,
                                                                 x = x_train_1000, 
                                                                 y = y_train_1000, 
                                                                 val_data = (x_val_1000, y_val_1000))
    save_all(filepath = "general_results", model_arch = "Aug_Alt_double_1000_auROC", curr_seed = curr_seed,
             callback = auroc_callback, model = model, val_data = (x_val_1000, y_val_1000))
    save_all(filepath = "general_results", model_arch = "Aug_Alt_double_1000_loss", curr_seed = curr_seed,
             callback = early_stopping_callback, model = model, val_data = (x_val_1000, y_val_1000))

Train on 34284 samples, validate on 12858 samples
Epoch 1/200
34284/34284 [==============================] - 3s 94us/step - loss: 0.7744 - acc: 0.5952 - val_loss: 0.6875 - val_acc: 0.6327
Epoch 2/200
34284/34284 [==============================] - 2s 45us/step - loss: 0.6692 - acc: 0.6343 - val_loss: 0.6775 - val_acc: 0.6243
Epoch 3/200
34284/34284 [==============================] - 2s 45us/step - loss: 0.6536 - acc: 0.6434 - val_loss: 0.6692 - val_acc: 0.6371
Epoch 4/200
34284/34284 [==============================] - 2s 45us/step - loss: 0.6424 - acc: 0.6516 - val_loss: 0.6634 - val_acc: 0.6284
Epoch 5/200
34284/34284 [==============================] - 2s 45us/step - loss: 0.6314 - acc: 0.6602 - val_loss: 0.6548 - val_acc: 0.6408
Epoch 6/200
34284/34284 [==============================] - 2s 45us/step - loss: 0.6226 - acc: 0.6653 - val_loss: 0.6520 - val_acc: 0.6504
Epoch 7/200
34284/34284 [==============================] - 2s 45us/step - loss: 0.6129 - acc: 0.6721 - val_loss: 0.6425 - 

In [ ]:
#seq length 200 auROC callback vs loss callback for auROC
#seq length 200 auROC callback vs loss callback for auPRC
#seq length 1000 auROC callback vs loss callback for auROC
#seq length 1000 auROC callback vs loss callback for auPRC

In [ ]:
def get_results_from_file(all_seeds, seq_len): 
    auROC_auROC_results = {
        'RC': [],
        'Siamese': [],
        'Aug': [], 
        'Aug_Alt': [],
        'Aug_Alt_double': [], 
        'Standard': []
    }
    auROC_auPRC_results = {
        'RC': [],
        'Siamese': [],
        'Aug': [], 
        'Aug_Alt': [],
        'Aug_Alt_double': [], 
        'Standard': []
    }
    
    loss_auROC_results = {
        'RC': [],
        'Siamese': [],
        'Aug': [], 
        'Aug_Alt': [],
        'Aug_Alt_double': [], 
        'Standard': []
    }
    loss_auPRC_results = {
        'RC': [],
        'Siamese': [],
        'Aug': [], 
        'Aug_Alt': [],
        'Aug_Alt_double': [], 
        'Standard': []
    }
    
    for curr_seed in all_seeds:
        for model_name in auROC_auROC_results: 
            f = open("general_results/%s/%s_%s_auROC.txt" % (str(curr_seed), model_name, str(seq_len)))
            lines = f.readlines()  
            auROC_auROC_results[model_name].append(float(lines[0][7:-1]))
            auROC_auPRC_results[model_name].append(float(lines[1][7:-1]))
            f.close()
            
            f = open("general_results/%s/%s_%s_loss.txt" % (str(curr_seed), model_name, str(seq_len)))
            lines = f.readlines()  
            loss_auROC_results[model_name].append(float(lines[0][7:-1]))
            loss_auPRC_results[model_name].append(float(lines[1][7:-1]))
            f.close()

    return auROC_auROC_result, auROC_auPRC_results, loss_auROC_results, loss_auPRC_results

In [ ]:
def get_stacked_graphs(result_1, result_2, y_min, y_max, metric, seq_len, save_model_name):
    result1_means = []
    result1_std = []
    
    result2_means = []
    result2_std = []
    
    N = len(result_1)/2
    for model_name in result_1: 
        result1_means.append(np.mean(result1_means[model_name]))
        result1_std.append(np.std(result1_means[model_name]))
    for model_name in result_2: 
        result2_means.append(np.mean(result2_means[model_name]))
        result2_std.append(np.std(result2_means[model_name]))

    ind = np.arange(N)
    width = 0.32
    
    fig, ax = plt.subplots()
    fig.set_size_inches(14, 8)
    plt.ylim([y_min,y_max])
    rects1 = ax.bar(ind, result1_means, width, color='r', yerr = result1_std, capsize = 5)
    rects2 = ax.bar(ind + width, result2_means, width, color='g', yerr = result2_std, capsize = 5)

    ax.set_ylabel(metric)
    ax.set_xlabel('Model Arch')
    ax.set_title('Median %s %s' % (metric, str(seq_len))
    ax.set_xticks(ind + width / 2)
    ax.set_xticklabels(('RC', 'Siamese', 'Aug', 'Aug_Alt', 'Aug_Alt_double', 'Standard'))
    
    ax.legend((rects1[0], rects2[0]), ('auROC', 'loss'))
    plt.show
    if save_model_name != None:
        plt.savefig(save_model_name + ".png")

In [ ]:
auROC_auROC_result_200, auROC_auPRC_results_200, loss_auROC_results_200, loss_auPRC_results_200 = get_results_from_file(all_seeds = all_seeds, seq_len = 200)
auROC_auROC_result_1000, auROC_auPRC_results_1000, loss_auROC_results_1000, loss_auPRC_results_1000 = get_results_from_file(all_seeds = all_seeds, seq_len = 1000)

In [ ]:
get_stacked_graphs(result_1 = auROC_auROC_result_200, result_2 = loss_auROC_results_200, y_min = 0.9, y_max = 1, metric = "auROC", seq_len = 200, save_model_name = None)
get_stacked_graphs(result_1 = auROC_auPRC_result_200, result_2 = loss_auPRC_results_200, y_min = 0.9, y_max = 1, metric = "auPRC", seq_len = 200, save_model_name = None)

In [ ]:
get_stacked_graphs(result_1 = auROC_auROC_result_1000, result_2 = loss_auROC_results_1000, y_min = 0.9, y_max = 1, metric = "auROC", seq_len = 1000, save_model_name = None)
get_stacked_graphs(result_1 = auROC_auPRC_result_1000, result_2 = loss_auPRC_results_1000, y_min = 0.9, y_max = 1, metric = "auPRC", seq_len = 1000, save_model_name = None)

In [31]:
for curr_seed in all_seeds: 
    model, early_stopping_callback, auroc_callback = train_model(model_wrapper = RC_WRAPPER, 
                                                                 aug = None, 
                                                                 curr_seed = curr_seed, 
                                                                 batch_size = 500,
                                                                 x = x_train_1000, 
                                                                 y = y_train_1000, 
                                                                 val_data = (x_val_1000, y_val_1000))
    save_all(filepath = "general_results", model_arch = "test_RC_1000_auROC", curr_seed = curr_seed,
             callback = auroc_callback, model = model, val_data = (x_val_1000, y_val_1000))
    save_all(filepath = "general_results", model_arch = "test_RC_1000_loss", curr_seed = curr_seed,
             callback = early_stopping_callback, model = model, val_data = (x_test_1000, y_test_1000))

Train on 17142 samples, validate on 12858 samples
Epoch 1/200
17142/17142 [==============================] - 4s 212us/step - loss: 0.7711 - acc: 0.5954 - val_loss: 0.6925 - val_acc: 0.6459
Epoch 2/200
17142/17142 [==============================] - 1s 54us/step - loss: 0.6747 - acc: 0.6305 - val_loss: 0.6779 - val_acc: 0.6221
Epoch 3/200
17142/17142 [==============================] - 1s 60us/step - loss: 0.6618 - acc: 0.6371 - val_loss: 0.6738 - val_acc: 0.6421
Epoch 4/200
17142/17142 [==============================] - 1s 59us/step - loss: 0.6512 - acc: 0.6447 - val_loss: 0.6706 - val_acc: 0.6253
Epoch 5/200
17142/17142 [==============================] - 1s 56us/step - loss: 0.6407 - acc: 0.6520 - val_loss: 0.6622 - val_acc: 0.6323
Epoch 6/200
17142/17142 [==============================] - 1s 61us/step - loss: 0.6312 - acc: 0.6581 - val_loss: 0.6534 - val_acc: 0.6454
Epoch 7/200
17142/17142 [==============================] - 1s 59us/step - loss: 0.6205 - acc: 0.6659 - val_loss: 0.6460 -

KeyboardInterrupt: 

In [11]:
for curr_seed in all_seeds: 
    model, early_stopping_callback, auroc_callback = train_model(model_wrapper = REG_WRAPPER, 
                                                                 aug = None, 
                                                                 curr_seed = curr_seed, 
                                                                 batch_size = 500,
                                                                 x = x_train_1000, 
                                                                 y = y_train_1000, 
                                                                 val_data = (x_val_1000, y_val_1000))
    save_all(filepath = "general_results", model_arch = "Standard_1000_auROC", curr_seed = curr_seed,
             callback = auroc_callback, model = model, val_data = (x_val_1000, y_val_1000))
    save_all(filepath = "general_results", model_arch = "Standard_1000_loss", curr_seed = curr_seed,
             callback = early_stopping_callback, model = model, val_data = (x_val_1000, y_val_1000))
    
    model, early_stopping_callback, auroc_callback = train_model(model_wrapper = REG_WRAPPER, 
                                                                 aug = "rev_after_each", 
                                                                 curr_seed = curr_seed, 
                                                                 batch_size = 500,
                                                                 x = x_train_1000, 
                                                                 y = y_train_1000, 
                                                                 val_data = (x_val_1000, y_val_1000))
    save_all(filepath = "general_results", model_arch = "Aug_1000_auROC", curr_seed = curr_seed,
             callback = auroc_callback, model = model, val_data = (x_val_1000, y_val_1000))
    save_all(filepath = "general_results", model_arch = "Aug_1000_loss", curr_seed = curr_seed,
             callback = early_stopping_callback, model = model, val_data = (x_val_1000, y_val_1000))
    
    model, early_stopping_callback, auroc_callback = train_model(model_wrapper = REG_WRAPPER, 
                                                                 aug = "rev_after_all", 
                                                                 curr_seed = curr_seed, 
                                                                 batch_size = 500,
                                                                 x = x_train_1000, 
                                                                 y = y_train_1000, 
                                                                 val_data = (x_val_1000, y_val_1000))
    save_all(filepath = "general_results", model_arch = "Aug_Alt_1000_auROC", curr_seed = curr_seed,
             callback = auroc_callback, model = model, val_data = (x_val_1000, y_val_1000))
    save_all(filepath = "general_results", model_arch = "Aug_Alt_1000_loss", curr_seed = curr_seed,
             callback = early_stopping_callback, model = model, val_data = (x_val_1000, y_val_1000))

Train on 17142 samples, validate on 12858 samples
Epoch 1/200
17142/17142 [==============================] - 2s 136us/step - loss: 0.7711 - acc: 0.5954 - val_loss: 0.6925 - val_acc: 0.6459
Epoch 2/200
17142/17142 [==============================] - 1s 57us/step - loss: 0.6747 - acc: 0.6304 - val_loss: 0.6779 - val_acc: 0.6221
Epoch 3/200
17142/17142 [==============================] - 1s 57us/step - loss: 0.6618 - acc: 0.6369 - val_loss: 0.6738 - val_acc: 0.6424
Epoch 4/200
17142/17142 [==============================] - 1s 56us/step - loss: 0.6513 - acc: 0.6448 - val_loss: 0.6706 - val_acc: 0.6247
Epoch 5/200
17142/17142 [==============================] - 1s 55us/step - loss: 0.6407 - acc: 0.6518 - val_loss: 0.6622 - val_acc: 0.6330
Epoch 6/200
17142/17142 [==============================] - 1s 56us/step - loss: 0.6312 - acc: 0.6576 - val_loss: 0.6534 - val_acc: 0.6453
Epoch 7/200
17142/17142 [==============================] - 1s 55us/step - loss: 0.6206 - acc: 0.6653 - val_loss: 0.6459 -

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



34284/34284 [==============================] - 2s 46us/step - loss: 0.3761 - acc: 0.8306 - val_loss: 0.4198 - val_acc: 0.8042
Epoch 62/200
34284/34284 [==============================] - 2s 45us/step - loss: 0.3736 - acc: 0.8329 - val_loss: 0.4170 - val_acc: 0.8055
Epoch 63/200
34284/34284 [==============================] - 2s 46us/step - loss: 0.3744 - acc: 0.8315 - val_loss: 0.4227 - val_acc: 0.8024
Epoch 64/200
34284/34284 [==============================] - 2s 46us/step - loss: 0.3738 - acc: 0.8327 - val_loss: 0.4218 - val_acc: 0.8016
Epoch 65/200
34284/34284 [==============================] - 2s 47us/step - loss: 0.3740 - acc: 0.8324 - val_loss: 0.4180 - val_acc: 0.8060
Epoch 66/200
34284/34284 [==============================] - 2s 47us/step - loss: 0.3742 - acc: 0.8311 - val_loss: 0.4179 - val_acc: 0.8035
Epoch 67/200
34284/34284 [==============================] - 2s 47us/step - loss: 0.3713 - acc: 0.8334 - val_loss: 0.4193 - val_acc: 0.8038
Epoch 68/200
34284/34284 [==============

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



34284/34284 [==============================] - 2s 46us/step - loss: 0.5364 - acc: 0.7294 - val_loss: 0.5581 - val_acc: 0.7136
Epoch 10/200
34284/34284 [==============================] - 2s 47us/step - loss: 0.5232 - acc: 0.7390 - val_loss: 0.5522 - val_acc: 0.7196
Epoch 11/200
34284/34284 [==============================] - 2s 47us/step - loss: 0.5141 - acc: 0.7456 - val_loss: 0.5475 - val_acc: 0.7252
Epoch 12/200
34284/34284 [==============================] - 2s 47us/step - loss: 0.5022 - acc: 0.7528 - val_loss: 0.5459 - val_acc: 0.7282
Epoch 13/200
34284/34284 [==============================] - 2s 47us/step - loss: 0.4946 - acc: 0.7583 - val_loss: 0.5240 - val_acc: 0.7328
Epoch 14/200
34284/34284 [==============================] - 2s 46us/step - loss: 0.4862 - acc: 0.7641 - val_loss: 0.5339 - val_acc: 0.7315
Epoch 15/200
34284/34284 [==============================] - 2s 47us/step - loss: 0.4795 - acc: 0.7673 - val_loss: 0.5199 - val_acc: 0.7429
Epoch 16/200
34284/34284 [==============

...

34284/34284 [==============================] - 2s 47us/step - loss: 0.3852 - acc: 0.8282 - val_loss: 0.4462 - val_acc: 0.7861
Epoch 33/200
34284/34284 [==============================] - 2s 48us/step - loss: 0.3824 - acc: 0.8287 - val_loss: 0.4426 - val_acc: 0.7886
Epoch 34/200
34284/34284 [==============================] - 2s 47us/step - loss: 0.3803 - acc: 0.8313 - val_loss: 0.4454 - val_acc: 0.7865
Epoch 35/200
34284/34284 [==============================] - 2s 47us/step - loss: 0.3815 - acc: 0.8287 - val_loss: 0.4705 - val_acc: 0.7701
Epoch 36/200
34284/34284 [==============================] - 2s 47us/step - loss: 0.3788 - acc: 0.8307 - val_loss: 0.4501 - val_acc: 0.7846
Epoch 37/200
34284/34284 [==============================] - 2s 47us/step - loss: 0.3796 - acc: 0.8308 - val_loss: 0.4739 - val_acc: 0.7710
Epoch 38/200
34284/34284 [==============================] - 2s 48us/step - loss: 0.3743 - acc: 0.8328 - val_loss: 0.4458 - val_acc: 0.7876
Epoch 39/200
34284/34284 [==============

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Epoch 49/200
17142/17142 [==============================] - 1s 55us/step - loss: 0.4268 - acc: 0.8007 - val_loss: 0.4806 - val_acc: 0.7632
Epoch 50/200
17142/17142 [==============================] - 1s 56us/step - loss: 0.4265 - acc: 0.8005 - val_loss: 0.4801 - val_acc: 0.7641
Epoch 51/200
17142/17142 [==============================] - 1s 56us/step - loss: 0.4238 - acc: 0.8031 - val_loss: 0.4804 - val_acc: 0.7618
Epoch 52/200
17142/17142 [==============================] - 1s 56us/step - loss: 0.4222 - acc: 0.8034 - val_loss: 0.4819 - val_acc: 0.7638
Epoch 53/200
17142/17142 [==============================] - 1s 56us/step - loss: 0.4197 - acc: 0.8051 - val_loss: 0.4769 - val_acc: 0.7632
Epoch 54/200
17142/17142 [==============================] - 1s 56us/step - loss: 0.4147 - acc: 0.8095 - val_loss: 0.4759 - val_acc: 0.7675
Epoch 55/200
17142/17142 [==============================] - 1s 55us/step - loss: 0.4139 - acc: 0.8082 - val_loss: 0.4756 - val_acc: 0.7667
Epoch 56/200
17142/17142 [=

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



34284/34284 [==============================] - 2s 52us/step - loss: 0.3823 - acc: 0.8279 - val_loss: 0.4203 - val_acc: 0.8038
Epoch 53/200
34284/34284 [==============================] - 2s 52us/step - loss: 0.3825 - acc: 0.8275 - val_loss: 0.4200 - val_acc: 0.8040
Epoch 54/200
34284/34284 [==============================] - 2s 52us/step - loss: 0.3815 - acc: 0.8280 - val_loss: 0.4168 - val_acc: 0.8053
Epoch 55/200
34284/34284 [==============================] - 2s 52us/step - loss: 0.3805 - acc: 0.8278 - val_loss: 0.4170 - val_acc: 0.8050
Epoch 56/200
34284/34284 [==============================] - 2s 51us/step - loss: 0.3801 - acc: 0.8285 - val_loss: 0.4218 - val_acc: 0.8029
Epoch 57/200
34284/34284 [==============================] - 2s 51us/step - loss: 0.3792 - acc: 0.8284 - val_loss: 0.4265 - val_acc: 0.7995
Epoch 58/200
34284/34284 [==============================] - 2s 52us/step - loss: 0.3800 - acc: 0.8286 - val_loss: 0.4199 - val_acc: 0.8029
Epoch 59/200
34284/34284 [==============

In [12]:
for curr_seed in all_seeds: 
    model, early_stopping_callback, auroc_callback = train_model(model_wrapper = SIAMESE_WRAPPER, 
                                                                 aug = None, 
                                                                 curr_seed = curr_seed, 
                                                                 batch_size = 500,
                                                                 x = x_train_1000, 
                                                                 y = y_train_1000, 
                                                                 val_data = (x_val_1000, y_val_1000))
    save_all(filepath = "general_results", model_arch = "Siamese_1000_auROC", curr_seed = curr_seed,
             callback = auroc_callback, model = model, val_data = (x_val_1000, y_val_1000))
    save_all(filepath = "general_results", model_arch = "Siamese_1000_loss", curr_seed = curr_seed,
             callback = early_stopping_callback, model = model, val_data = (x_val_1000, y_val_1000))

Train on 17142 samples, validate on 12858 samples
Epoch 1/200
17142/17142 [==============================] - 7s 407us/step - loss: 0.7456 - acc: 0.6061 - val_loss: 0.6687 - val_acc: 0.6595
Epoch 2/200
17142/17142 [==============================] - 1s 80us/step - loss: 0.6530 - acc: 0.6487 - val_loss: 0.6540 - val_acc: 0.6437
Epoch 3/200
17142/17142 [==============================] - 1s 79us/step - loss: 0.6429 - acc: 0.6543 - val_loss: 0.6501 - val_acc: 0.6588
Epoch 4/200
17142/17142 [==============================] - 1s 79us/step - loss: 0.6327 - acc: 0.6611 - val_loss: 0.6453 - val_acc: 0.6488
Epoch 5/200
17142/17142 [==============================] - 1s 79us/step - loss: 0.6233 - acc: 0.6671 - val_loss: 0.6396 - val_acc: 0.6468
Epoch 6/200
17142/17142 [==============================] - 1s 81us/step - loss: 0.6143 - acc: 0.6708 - val_loss: 0.6285 - val_acc: 0.6645
Epoch 7/200
17142/17142 [==============================] - 1s 79us/step - loss: 0.6029 - acc: 0.6795 - val_loss: 0.6186 -

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



17142/17142 [==============================] - 1s 81us/step - loss: 0.3297 - acc: 0.8589 - val_loss: 0.4061 - val_acc: 0.8117
Epoch 116/200
17142/17142 [==============================] - 1s 80us/step - loss: 0.3318 - acc: 0.8559 - val_loss: 0.3942 - val_acc: 0.8168
Epoch 117/200
17142/17142 [==============================] - 1s 81us/step - loss: 0.3295 - acc: 0.8583 - val_loss: 0.3868 - val_acc: 0.8213
Epoch 118/200
17142/17142 [==============================] - 1s 80us/step - loss: 0.3269 - acc: 0.8590 - val_loss: 0.3904 - val_acc: 0.8192
Epoch 119/200
17142/17142 [==============================] - 1s 81us/step - loss: 0.3299 - acc: 0.8580 - val_loss: 0.3923 - val_acc: 0.8195
Epoch 120/200
17142/17142 [==============================] - 1s 82us/step - loss: 0.3255 - acc: 0.8612 - val_loss: 0.3859 - val_acc: 0.8216
Epoch 121/200
17142/17142 [==============================] - 1s 80us/step - loss: 0.3245 - acc: 0.8616 - val_loss: 0.3873 - val_acc: 0.8210
Epoch 122/200
17142/17142 [=======

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



17142/17142 [==============================] - 1s 79us/step - loss: 0.3502 - acc: 0.8505 - val_loss: 0.3986 - val_acc: 0.8154
Epoch 86/200
17142/17142 [==============================] - 1s 79us/step - loss: 0.3520 - acc: 0.8482 - val_loss: 0.3991 - val_acc: 0.8154
Epoch 87/200
17142/17142 [==============================] - 1s 79us/step - loss: 0.3548 - acc: 0.8473 - val_loss: 0.4008 - val_acc: 0.8143
Epoch 88/200
17142/17142 [==============================] - 1s 80us/step - loss: 0.3535 - acc: 0.8471 - val_loss: 0.4087 - val_acc: 0.8096
Epoch 89/200
17142/17142 [==============================] - 1s 79us/step - loss: 0.3523 - acc: 0.8463 - val_loss: 0.4097 - val_acc: 0.8095
Epoch 90/200
17142/17142 [==============================] - 1s 79us/step - loss: 0.3537 - acc: 0.8462 - val_loss: 0.4145 - val_acc: 0.8064
Epoch 91/200
17142/17142 [==============================] - 1s 79us/step - loss: 0.3490 - acc: 0.8486 - val_loss: 0.3954 - val_acc: 0.8173
Epoch 92/200
17142/17142 [==============

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



17142/17142 [==============================] - 1s 81us/step - loss: 0.3248 - acc: 0.8613 - val_loss: 0.4163 - val_acc: 0.8055
Epoch 137/200
17142/17142 [==============================] - 1s 81us/step - loss: 0.3291 - acc: 0.8573 - val_loss: 0.4207 - val_acc: 0.8059
Epoch 138/200
17142/17142 [==============================] - 1s 82us/step - loss: 0.3250 - acc: 0.8607 - val_loss: 0.3932 - val_acc: 0.8186
Epoch 139/200
17142/17142 [==============================] - 1s 82us/step - loss: 0.3212 - acc: 0.8632 - val_loss: 0.3961 - val_acc: 0.8178
auroc: 0.8832150869028016
auprc: 0.8101884025260963
auroc: 0.8819873501594798
auprc: 0.8084458521069542
Train on 17142 samples, validate on 12858 samples
Epoch 1/200
17142/17142 [==============================] - 9s 514us/step - loss: 0.6846 - acc: 0.6226 - val_loss: 0.6677 - val_acc: 0.6441
Epoch 2/200
17142/17142 [==============================] - 1s 82us/step - loss: 0.6529 - acc: 0.6480 - val_loss: 0.6559 - val_acc: 0.6464
Epoch 3/200
17142/17142